In [2]:
# Packages
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account

# Authenticate GCP/BQ and connect
credentials = service_account.Credentials.from_service_account_file(os.environ['gcp_credentials'])
project_id = 'freightwaves-data-science'
client = bigquery.Client(credentials=credentials, project=project_id)

## What are the most common accessorial charges?
It's hard to evaluate the different accessorial charge types because there are 10,188 different ones (many of which are different ways to say the same thing), but fuel, base, and discount charges are by far the most commonly occurring charges (by a lot - well over 10 million occurrences for each, but closer to 100 million for fuel and base charges). After that, you're looking at freight, program fees, stop off, tolls, and value added / goods & service taxes as the remaining accessorial charges with over 1 million occurrences.

In [29]:
# List of accessorial charges, ordered by number of occurrences
query_job = client.query("""
    select
      accessorial_charge_description
      ,count(accessorial_charge_description) count
    from `freightwaves-data-factory.warehouse.beetlejuice`
    group by accessorial_charge_description
    order by count(accessorial_charge_description) desc
    """)

query = query_job.result().to_dataframe()
query

,accessorial_charge_description,count
0,FUEL SURCHARGE,75611181
1,BASE CHARGE,25367631
2,FUEL CHARGE,20098528
3,DISCOUNT,11974635
4,BASE CHARGE ...,9760971
...,...,...
10197,ADDRESS CORRECTION ...,1
10198,FAF/FUEL ADJUSTMENT FACTOR,1
10199,SMETA AUDIT,1
10200,COVID-19 SURCHARGE,1


## What loads have accessorial charges?
92% of global loads have accessorial (non-base) charges, while over 99% of legacy loads have them.

Among the 22 transportation types with over 10000 loads, truckload (flatbed), courier, and ocean (broker/forwarder) have the fewest loads with accessorial charges at 60%, 68%, and 87%. Truckload (expedited), LTL, ocean, truckload (bulk), air freight, air freight (broker/forwarder), and cust pickup/expense all had over 99% loads with accessorial charges.

Amount the 222 state lanes with over 100000 loads, all but five have 95%+ loads with accessorial charges. The five that do not are TN-TN, IA-NE, MS-TX, NC-NC, and OR-OR. If we lower the threshold to lanes with over 10000 loads, then seven lanes with less than 70% accessorial charges appear: HI-HI, WV-WV, GJ-GJ, TM-TX, UT-UT, CO-IL, and EM-TX.

In [19]:
# Percentage of all loads that have accessorial charges besides base
query_job = client.query("""
with all_loads as (
  select
    upper(system_type) as system_type
    ,count(distinct cass_shipment_id) as all_loads
  from `freightwaves-data-factory.warehouse.beetlejuice`
  group by upper(system_type)
),
loads_with_true_accessorial as (
  select
    upper(system_type) as system_type
    ,count(distinct cass_shipment_id) as loads_with_true_accessorial
  from `freightwaves-data-factory.warehouse.beetlejuice`
    where upper(accessorial_charge_description) not like "%BASE%"
  group by upper(system_type)
)

select
  all_loads.system_type
  ,true_accessorial.loads_with_true_accessorial
  ,all_loads.all_loads
  ,round(safe_divide(true_accessorial.loads_with_true_accessorial, all_loads.all_loads)*100, 2) as pct_loads_with_accessorial
from all_loads
left join loads_with_true_accessorial as true_accessorial
  on all_loads.system_type = true_accessorial.system_type
""")

query = query_job.result().to_dataframe()
query.head()


,system_type,loads_with_true_accessorial,all_loads,pct_loads_with_accessorial
0,LEGACY,79576016,79660516,99.89
1,GLOBAL,31595036,34383044,91.89


In [20]:
# Percentage of all loads that have accessorial charges besides base, grouped by transportation mode
query_job = client.query("""
with all_loads as (
  select
    transportation_mode_description
    ,count(distinct cass_shipment_id) as all_loads
  from `freightwaves-data-factory.warehouse.beetlejuice`
  group by transportation_mode_description
),
loads_with_true_accessorial as (
  select
    transportation_mode_description
    ,count(distinct cass_shipment_id) as loads_with_true_accessorial
  from `freightwaves-data-factory.warehouse.beetlejuice`
    where upper(accessorial_charge_description) not like "%BASE%"
  group by transportation_mode_description
)

select
  all_loads.transportation_mode_description
  ,true_accessorial.loads_with_true_accessorial
  ,all_loads.all_loads
  ,round(safe_divide(true_accessorial.loads_with_true_accessorial, all_loads.all_loads)*100, 2) as pct_loads_with_accessorial
from all_loads
left join loads_with_true_accessorial as true_accessorial
  on all_loads.transportation_mode_description = true_accessorial.transportation_mode_description
where true_accessorial.loads_with_true_accessorial > 10000
order by safe_divide(true_accessorial.loads_with_true_accessorial, all_loads)
""")

query = query_job.result().to_dataframe()
query


,transportation_mode_description,loads_with_true_accessorial,all_loads,pct_loads_with_accessorial
0,TRUCKLOAD (FLATBED),21769,36004,60.46
1,COURIER,468663,688145,68.11
2,OCEAN (BROKER/FORWARDER),43759,50278,87.03
3,INTERMODAL,2440699,2663103,91.65
4,AIR FREIGHT (EXPRESS),1461469,1578397,92.59
5,DRAYAGE,156423,166619,93.88
6,TRUCKLOAD,1266626,1334697,94.90
7,TRUCKLOAD (BULK - LIQUID),210920,219234,96.21
8,RAIL,1350565,1391250,97.08
9,BROKER,242963,249863,97.24


In [22]:
# Percentage of all loads that have accessorial charges besides base, grouped by lane
query_job = client.query("""
with all_loads as (
  select
    concat(origin_state, "-", destination_state) as state_lane
    ,count(distinct cass_shipment_id) as all_loads
  from `freightwaves-data-factory.warehouse.beetlejuice`
  group by concat(origin_state, "-", destination_state)
),
loads_with_true_accessorial as (
  select
    concat(origin_state, "-", destination_state) as state_lane
    ,count(distinct cass_shipment_id) as loads_with_true_accessorial
  from `freightwaves-data-factory.warehouse.beetlejuice`
    where upper(accessorial_charge_description) not like "%BASE%"
  group by concat(origin_state, "-", destination_state)
)

select
  all_loads.state_lane
  ,true_accessorial.loads_with_true_accessorial
  ,all_loads.all_loads
  ,round(safe_divide(true_accessorial.loads_with_true_accessorial, all_loads.all_loads)*100, 2) as pct_loads_with_accessorial
from all_loads
left join loads_with_true_accessorial as true_accessorial
  on all_loads.state_lane = true_accessorial.state_lane
where true_accessorial.loads_with_true_accessorial > 100000
order by safe_divide(true_accessorial.loads_with_true_accessorial, all_loads)
""")

query = query_job.result().to_dataframe()
query


,state_lane,loads_with_true_accessorial,all_loads,pct_loads_with_accessorial
0,TN-TN,334466,383132,87.30
1,IA-NE,111725,121220,92.17
2,MS-TX,108655,117699,92.32
3,NC-NC,780881,838733,93.10
4,OR-OR,105365,113061,93.19
...,...,...,...,...
217,SK-SK,108368,108441,99.93
218,BC-BC,245999,246147,99.94
219,ME-ME,177865,177905,99.98
220,ME-NY,206155,206175,99.99


## How do different accessorial charges affect the variance of lanes?
NOTE: I tested these keywords that Julie suggested: toll, deadhead, labor, lumper, detention, CA/California, pickup, delivery, used, redelivery, linehaul, stop, & loading. I also tested tax, discount, freight, handling, security, weight, customs, and hazardous after they appeared in a word frequency search.

In [30]:
# Comparing variance in payment for each lane
# yes i know this is bad code i'll refactor it later ugh
query_job = client.query("""
with data_with_lanes as (
  select
    concat(origin_state, "-", destination_state) as state_lane
    ,upper(accessorial_charge_description) as accessorial_charge_description
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,max(amount_paid) over (partition by cass_shipment_id) as amount_paid
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where concat(origin_state, "-", destination_state) is not null
    -- and created_on > "2019-01-01"
),
-- take out all records with base or linehaul, then from the amount paid, subtract the sum of the remaining accessorial charges on a shipment-by-shipment basis. This equals your linehaul.
data_with_linehaul_prep1 as (
  select
    *
    ,amount_paid - sum(accessorial_charge_amount) over (partition by cass_shipment_id) as linehaul
  from data_with_lanes
  where accessorial_charge_description not like "%BASE%"
    and accessorial_charge_description not like "%LINEHAUL%"
    and accessorial_charge_description not like "%LINE HAUL%"
),
-- bring back the original dataset but add in the new column that has the calculated linehaul
data_with_linehaul_prep2 as (
  select
    lane.state_lane
    ,lane.accessorial_charge_description
    ,lane.accessorial_charge_amount
    ,lane.cass_shipment_id
    ,lane.system_type
    ,lane.amount_paid
    ,max(lh.linehaul) over (partition by lane.cass_shipment_id) as linehaul -- yeah it's weird but it works
  from data_with_lanes as lane
  left join data_with_linehaul_prep1 as lh
    on lane.cass_shipment_id = lh.cass_shipment_id
    and lane.accessorial_charge_description = lh.accessorial_charge_description
    and lane.accessorial_charge_amount = lh.accessorial_charge_amount
),
-- if linehaul is null, that's because that particular shipment ONLY had a base or linehaul charge. However, that means the linehaul is simply the amount paid for this shipment, so this takes care of that.
data_with_linehaul as (
  select
    state_lane
    ,accessorial_charge_description
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,amount_paid
    ,if(linehaul is null, amount_paid, linehaul) as linehaul
  from data_with_linehaul_prep2
),
-- linehaul
loads_with_linehaul as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as lh_loads_per_lane
    ,cass_shipment_id
    ,linehaul
  from data_with_linehaul
  group by state_lane, cass_shipment_id, linehaul
),
linehaul_variance as (
  select
    state_lane
    ,lh_loads_per_lane
    ,round(variance(linehaul), 2) as lh_variance
  from loads_with_linehaul
  group by state_lane, lh_loads_per_lane
),
-- this code is going to be repeated for every accessorial charge, and then the resulting tables will be unioned. To test new variables, copy and past the three CTEs below this and switch out 'fuel' for the new variable. (yes i know what DRY code is yes i know this is a bad way to do this don't come at me)
-- fuel
loads_with_lh_fuel as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_fuel_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%FUEL%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_fuel_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_fuel_per_load), 2) as acc_variance
  from loads_with_lh_fuel
  group by state_lane, acc_loads_per_lane
),
lh_vs_fuel_variance as (
  select 
    lh.state_lane
    ,'fuel' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,fuel.acc_loads_per_lane
    ,lh.lh_variance
    ,fuel.acc_variance
    ,round(safe_divide(lh.lh_variance, fuel.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_fuel_variance as fuel
    on lh.state_lane = fuel.state_lane
  where lh.lh_variance is not null
    and fuel.acc_variance is not null
    -- and fuel.acc_loads_per_lane > 10000
),
-- toll
loads_with_lh_toll as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_toll_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%TOLL%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_toll_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_toll_per_load), 2) as acc_variance
  from loads_with_lh_toll
  group by state_lane, acc_loads_per_lane
),
lh_vs_toll_variance as (
  select 
    lh.state_lane
    ,'toll' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,toll.acc_loads_per_lane
    ,lh.lh_variance
    ,toll.acc_variance
    ,round(safe_divide(lh.lh_variance, toll.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_toll_variance as toll
    on lh.state_lane = toll.state_lane
  where lh.lh_variance is not null
    and toll.acc_variance is not null
    -- and toll.acc_loads_per_lane > 1000
),
-- labor
loads_with_lh_labor as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_labor_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%LABOR%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_labor_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_labor_per_load), 2) as acc_variance
  from loads_with_lh_labor
  group by state_lane, acc_loads_per_lane
),
lh_vs_labor_variance as (
  select 
    lh.state_lane
    ,'labor' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,labor.acc_loads_per_lane
    ,lh.lh_variance
    ,labor.acc_variance
    ,round(safe_divide(lh.lh_variance, labor.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_labor_variance as labor
    on lh.state_lane = labor.state_lane
  where lh.lh_variance is not null
    and labor.acc_variance is not null
    -- and labor.acc_loads_per_lane > 1000
),
-- lumper
loads_with_lh_lumper as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_lumper_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%LUMPER%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_lumper_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_lumper_per_load), 2) as acc_variance
  from loads_with_lh_lumper
  group by state_lane, acc_loads_per_lane
),
lh_vs_lumper_variance as (
  select 
    lh.state_lane
    ,'lumper' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,lumper.acc_loads_per_lane
    ,lh.lh_variance
    ,lumper.acc_variance
    ,round(safe_divide(lh.lh_variance, lumper.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_lumper_variance as lumper
    on lh.state_lane = lumper.state_lane
  where lh.lh_variance is not null
    and lumper.acc_variance is not null
    -- and lumper.acc_loads_per_lane > 100
),
-- detention
loads_with_lh_detention as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_detention_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%DETENTION%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_detention_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_detention_per_load), 2) as acc_variance
  from loads_with_lh_detention
  group by state_lane, acc_loads_per_lane
),
lh_vs_detention_variance as (
  select 
    lh.state_lane
    ,'detention' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,detention.acc_loads_per_lane
    ,lh.lh_variance
    ,detention.acc_variance
    ,round(safe_divide(lh.lh_variance, detention.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_detention_variance as detention
    on lh.state_lane = detention.state_lane
  where lh.lh_variance is not null
    and detention.acc_variance is not null
    -- and detention.acc_loads_per_lane > 100
),
-- pickup & delivery
loads_with_lh_pickup_delivery as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_pickup_delivery_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%PICKUP%"
    or accessorial_charge_description like "%PICK UP%"
    or accessorial_charge_description like "%DELIVERY%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_pickup_delivery_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_pickup_delivery_per_load), 2) as acc_variance
  from loads_with_lh_pickup_delivery
  group by state_lane, acc_loads_per_lane
),
lh_vs_pickup_delivery_variance as (
  select 
    lh.state_lane
    ,'pickup/delivery' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,pd.acc_loads_per_lane
    ,lh.lh_variance
    ,pd.acc_variance
    ,round(safe_divide(lh.lh_variance, pd.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_pickup_delivery_variance as pd
    on lh.state_lane = pd.state_lane
  where lh.lh_variance is not null
    and pd.acc_variance is not null
    -- and pd.acc_loads_per_lane > 100
),
-- used
loads_with_lh_used as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_used_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%USED%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_used_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_used_per_load), 2) as acc_variance
  from loads_with_lh_used
  group by state_lane, acc_loads_per_lane
),
lh_vs_used_variance as (
  select 
    lh.state_lane
    ,'used' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,used.acc_loads_per_lane
    ,lh.lh_variance
    ,used.acc_variance
    ,round(safe_divide(lh.lh_variance, used.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_used_variance as used
    on lh.state_lane = used.state_lane
  where lh.lh_variance is not null
    and used.acc_variance is not null
    -- and used.acc_loads_per_lane > 10
),
-- redelivery
loads_with_lh_redelivery as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_redelivery_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%REDELIVERY%"
    or accessorial_charge_description like "%RE-DELIVERY%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_redelivery_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_redelivery_per_load), 2) as acc_variance
  from loads_with_lh_redelivery
  group by state_lane, acc_loads_per_lane
),
lh_vs_redelivery_variance as (
  select 
    lh.state_lane
    ,'redelivery' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,redelivery.acc_loads_per_lane
    ,lh.lh_variance
    ,redelivery.acc_variance
    ,round(safe_divide(lh.lh_variance, redelivery.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_redelivery_variance as redelivery
    on lh.state_lane = redelivery.state_lane
  where lh.lh_variance is not null
    and redelivery.acc_variance is not null
    -- and redelivery.acc_loads_per_lane > 10
),
-- stop
loads_with_lh_stop as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_stop_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%STOP%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_stop_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_stop_per_load), 2) as acc_variance
  from loads_with_lh_stop
  group by state_lane, acc_loads_per_lane
),
lh_vs_stop_variance as (
  select 
    lh.state_lane
    ,'stop' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,stop.acc_loads_per_lane
    ,lh.lh_variance
    ,stop.acc_variance
    ,round(safe_divide(lh.lh_variance, stop.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_stop_variance as stop
    on lh.state_lane = stop.state_lane
  where lh.lh_variance is not null
    and stop.acc_variance is not null
    -- and stop.acc_loads_per_lane > 100
),
-- loading
loads_with_lh_loading as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_loading_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%LOADING%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_loading_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_loading_per_load), 2) as acc_variance
  from loads_with_lh_loading
  group by state_lane, acc_loads_per_lane
),
lh_vs_loading_variance as (
  select 
    lh.state_lane
    ,'loading' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,loading.acc_loads_per_lane
    ,lh.lh_variance
    ,loading.acc_variance
    ,round(safe_divide(lh.lh_variance, loading.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_loading_variance as loading
    on lh.state_lane = loading.state_lane
  where lh.lh_variance is not null
    and loading.acc_variance is not null
    -- and loading.acc_loads_per_lane > 100
),
-- tax
loads_with_lh_tax as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_tax_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%TAX%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_tax_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_tax_per_load), 2) as acc_variance
  from loads_with_lh_tax
  group by state_lane, acc_loads_per_lane
),
lh_vs_tax_variance as (
  select 
    lh.state_lane
    ,'tax' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,tax.acc_loads_per_lane
    ,lh.lh_variance
    ,tax.acc_variance
    ,round(safe_divide(lh.lh_variance, tax.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_tax_variance as tax
    on lh.state_lane = tax.state_lane
  where lh.lh_variance is not null
    and tax.acc_variance is not null
    -- and tax.acc_loads_per_lane > 100
),
-- discount
loads_with_lh_discount as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_discount_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%DISCOUNT%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_discount_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_discount_per_load), 2) as acc_variance
  from loads_with_lh_discount
  group by state_lane, acc_loads_per_lane
),
lh_vs_discount_variance as (
  select 
    lh.state_lane
    ,'discount' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,discount.acc_loads_per_lane
    ,lh.lh_variance
    ,discount.acc_variance
    ,round(safe_divide(lh.lh_variance, discount.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_discount_variance as discount
    on lh.state_lane = discount.state_lane
  where lh.lh_variance is not null
    and discount.acc_variance is not null
    -- and discount.acc_loads_per_lane > 100
),
-- freight
loads_with_lh_freight as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_freight_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%FREIGHT%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_freight_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_freight_per_load), 2) as acc_variance
  from loads_with_lh_freight
  group by state_lane, acc_loads_per_lane
),
lh_vs_freight_variance as (
  select 
    lh.state_lane
    ,'freight' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,freight.acc_loads_per_lane
    ,lh.lh_variance
    ,freight.acc_variance
    ,round(safe_divide(lh.lh_variance, freight.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_freight_variance as freight
    on lh.state_lane = freight.state_lane
  where lh.lh_variance is not null
    and freight.acc_variance is not null
    -- and freight.acc_loads_per_lane > 100
),
-- handling
loads_with_lh_handling as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_handling_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%HANDLING%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_handling_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_handling_per_load), 2) as acc_variance
  from loads_with_lh_handling
  group by state_lane, acc_loads_per_lane
),
lh_vs_handling_variance as (
  select 
    lh.state_lane
    ,'handling' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,handling.acc_loads_per_lane
    ,lh.lh_variance
    ,handling.acc_variance
    ,round(safe_divide(lh.lh_variance, handling.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_handling_variance as handling
    on lh.state_lane = handling.state_lane
  where lh.lh_variance is not null
    and handling.acc_variance is not null
    -- and handling.acc_loads_per_lane > 100
),
-- security
loads_with_lh_security as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_security_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%SECURITY%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_security_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_security_per_load), 2) as acc_variance
  from loads_with_lh_security
  group by state_lane, acc_loads_per_lane
),
lh_vs_security_variance as (
  select 
    lh.state_lane
    ,'security' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,security.acc_loads_per_lane
    ,lh.lh_variance
    ,security.acc_variance
    ,round(safe_divide(lh.lh_variance, security.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_security_variance as security
    on lh.state_lane = security.state_lane
  where lh.lh_variance is not null
    and security.acc_variance is not null
    -- and security.acc_loads_per_lane > 100
),
-- weight
loads_with_lh_weight as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_weight_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%WEIGHT%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_weight_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_weight_per_load), 2) as acc_variance
  from loads_with_lh_weight
  group by state_lane, acc_loads_per_lane
),
lh_vs_weight_variance as (
  select 
    lh.state_lane
    ,'weight' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,weight.acc_loads_per_lane
    ,lh.lh_variance
    ,weight.acc_variance
    ,round(safe_divide(lh.lh_variance, weight.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_weight_variance as weight
    on lh.state_lane = weight.state_lane
  where lh.lh_variance is not null
    and weight.acc_variance is not null
    -- and weight.acc_loads_per_lane > 100
),
-- customs
loads_with_lh_customs as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_customs_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%CUSTOMS%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_customs_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_customs_per_load), 2) as acc_variance
  from loads_with_lh_customs
  group by state_lane, acc_loads_per_lane
),
lh_vs_customs_variance as (
  select 
    lh.state_lane
    ,'customs' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,customs.acc_loads_per_lane
    ,lh.lh_variance
    ,customs.acc_variance
    ,round(safe_divide(lh.lh_variance, customs.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_customs_variance as customs
    on lh.state_lane = customs.state_lane
  where lh.lh_variance is not null
    and customs.acc_variance is not null
    -- and customs.acc_loads_per_lane > 100
),
-- hazardous
loads_with_lh_hazardous as (
  select
    state_lane
    ,count(distinct cass_shipment_id) over (partition by state_lane) as acc_loads_per_lane
    ,cass_shipment_id
    ,linehaul + sum(accessorial_charge_amount) as lh_hazardous_per_load
  from data_with_linehaul
  where accessorial_charge_description like "%HAZARDOUS%"
  group by state_lane, cass_shipment_id, linehaul
),
lh_hazardous_variance as (
  select
    state_lane
    ,acc_loads_per_lane
    ,round(variance(lh_hazardous_per_load), 2) as acc_variance
  from loads_with_lh_hazardous
  group by state_lane, acc_loads_per_lane
),
lh_vs_hazardous_variance as (
  select 
    lh.state_lane
    ,'hazardous' as accessorial_charge
    ,lh.lh_loads_per_lane
    ,hazardous.acc_loads_per_lane
    ,lh.lh_variance
    ,hazardous.acc_variance
    ,round(safe_divide(lh.lh_variance, hazardous.acc_variance), 2) as variance_decrease_pct
  from linehaul_variance as lh
  left join lh_hazardous_variance as hazardous
    on lh.state_lane = hazardous.state_lane
  where lh.lh_variance is not null
    and hazardous.acc_variance is not null
    -- and hazardous.acc_loads_per_lane > 100
),
-- union all that ish
unioned as (
  select * from lh_vs_fuel_variance
  union all
  select * from lh_vs_toll_variance
  union all
  select * from lh_vs_labor_variance
  union all
  select * from lh_vs_lumper_variance
  union all
  select * from lh_vs_detention_variance
  union all
  select * from lh_vs_pickup_delivery_variance
  union all
  select * from lh_vs_used_variance
  union all
  select * from lh_vs_redelivery_variance
  union all
  select * from lh_vs_stop_variance
  union all
  select * from lh_vs_loading_variance
  union all
  select * from lh_vs_tax_variance
  union all
  select * from lh_vs_discount_variance
  union all
  select * from lh_vs_freight_variance
  union all
  select * from lh_vs_handling_variance
  union all
  select * from lh_vs_security_variance
  union all
  select * from lh_vs_weight_variance
  union all
  select * from lh_vs_customs_variance
  union all
  select * from lh_vs_hazardous_variance
)

select
  accessorial_charge
  ,avg(lh_variance) as lh_variance_avg
  ,avg(acc_variance) as acc_variance_avg
  ,round(((avg(acc_variance)-avg(lh_variance))/avg(lh_variance))*100, 2) as variance_avg_pct_diff
from unioned
where acc_loads_per_lane > 1000
group by accessorial_charge
order by ((avg(acc_variance)-avg(lh_variance))/avg(lh_variance))*100
""")

query = query_job.result().to_dataframe()
query

,accessorial_charge,lh_variance_avg,acc_variance_avg,variance_avg_pct_diff
0,redelivery,1.603439e+06,3.673928e+05,-77.09
1,hazardous,2.264544e+06,9.135819e+05,-59.66
2,pickup/delivery,3.461260e+06,1.414448e+06,-59.13
3,loading,2.523799e+06,1.226861e+06,-51.39
4,used,1.486482e+06,7.910070e+05,-46.79
5,stop,4.795727e+06,2.720437e+06,-43.27
6,lumper,2.058950e+06,1.316861e+06,-36.04
7,weight,2.860565e+06,1.840939e+06,-35.64
8,toll,1.226237e+07,8.848265e+06,-27.84
9,fuel,4.674347e+06,3.513121e+06,-24.84


## How are accessorial charge types distributed across legacy vs global?
Among those 10 variables, their distribution doesn't change much between legacy and global, but pickup/delivery sees a ~5% decrease and fuel sees a ~5.5% increase.

In [12]:
# Comparing accessorial charge description by system type
query_job = client.query("""
with unioned as (
  select
    upper(system_type) as system_type
    ,"FUEL" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%FUEL%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"TOLL" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%TOLL%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"LABOR" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%LABOR%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"LUMPER" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%LUMPER%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"DETENTION" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%DETENTION%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"PICKUP/DELIVERY" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%PICKUP%"
    or upper(accessorial_charge_description) like "%PICK UP%"
    or upper(accessorial_charge_description) like "%DELIVERY%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"USED" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%USED%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"REDELIVERY" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%REDELIVERY%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"STOP" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%STOP%"
  group by upper(system_type)

  union all

  select
    upper(system_type) as system_type
    ,"LOADING" as accessorial_charge_description
    ,count(*) as count
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where upper(accessorial_charge_description) like "%LOADING%"
  group by upper(system_type)
  order by accessorial_charge_description, system_type
),
wide as (
  select
    accessorial_charge_description
    ,max(if( system_type='LEGACY', count, NULL ) ) AS legacy
    ,max(if( system_type='GLOBAL', count, NULL ) ) AS global
  from unioned
  group by accessorial_charge_description
  order by accessorial_charge_description
),
sum as (
  select
    *
    ,sum(legacy) over () as legacy_sum
    ,sum(global) over () as global_sum
  from wide
),
pct as (
  select
    accessorial_charge_description
    ,legacy
    ,legacy_sum
    ,global
    ,global_sum
    ,round((legacy/legacy_sum)*100, 2) as legacy_pct
    ,round((global/global_sum)*100, 2) as global_pct
    ,round(((legacy/legacy_sum)-(global/global_sum))*100, 2) as pct_diff
  from sum
)

select * from pct
order by pct_diff
""")

query = query_job.result().to_dataframe()
query

,accessorial_charge_description,legacy,legacy_sum,global,global_sum,legacy_pct,global_pct,pct_diff
0,PICKUP/DELIVERY,1190015,87561264,2054215,32944598,1.36,6.24,-4.88
1,TOLL,1686049,87561264,1423543,32944598,1.93,4.32,-2.40
2,LOADING,813333,87561264,373626,32944598,0.93,1.13,-0.21
3,REDELIVERY,61942,87561264,46121,32944598,0.07,0.14,-0.07
4,USED,44223,87561264,32121,32944598,0.05,0.10,-0.05
5,DETENTION,1048470,87561264,388657,32944598,1.20,1.18,0.02
6,LABOR,921688,87561264,165909,32944598,1.05,0.50,0.55
7,STOP,2268954,87561264,654066,32944598,2.59,1.99,0.61
8,LUMPER,752565,87561264,8618,32944598,0.86,0.03,0.83
9,FUEL,78774025,87561264,27797722,32944598,89.96,84.38,5.59


## How are accessorial charge amounts distributed?

Across all loads, base (linehaul) and fuel account for 91% of the charge payments; the other eight types account for 3.5%; all other charges account for 5.5%.

When examining the shippers with over 1,000,000 loads, we see some interesting distributions among the accessorial charge amounts. For most, base (linehaul) is the primary cost, though 0337, 6057, and 6463 register it as basically zero. Fuel is <10% for 1235, 0333, 1414, 1348, and 1336, but >90% for 0373 and 6055. Detention, loading, lumper, pickup/delivery, and used are neglible. Labor, toll and stop report one shipper each with >10%: 2212, 2026, and 6052, respectively.

When examining lanes with >500,000 loads, we find that base and fuel are the only significant accessorial charges out of the ten tested, often accounting for >90% of of the total charges. However, the 'other' charges seems to be really important here, as they range from -24% (likely due to a discount) all the way to 71%. This is an example of where it would be helpful to be able to categorize the accessorial charges, as there are so many of them it becomes extremely difficult to understand their impact.

In [31]:
# Comparing accessorial charge description by system type
query_job = client.query("""
with data_with_lanes as (
  select
    concat(origin_state, "-", destination_state) as state_lane
    ,upper(accessorial_charge_description) as accessorial_charge_description
    ,case
      when upper(accessorial_charge_description) like "%BASE%" then "BASE"
      when upper(accessorial_charge_description) like "%FUEL%" then "FUEL"
      when upper(accessorial_charge_description) like "%TOLL%" then "TOLL"
      when upper(accessorial_charge_description) like "%LABOR%" then "LABOR"
      when upper(accessorial_charge_description) like "%LUMPER%" then "LUMPER"
      when upper(accessorial_charge_description) like "%DETENTION%" then "DETENTION"
      when upper(accessorial_charge_description) like "%PICKUP%"
        or upper(accessorial_charge_description) like "%PICK UP%"
        or upper(accessorial_charge_description) like "%DELIVERY%" then "PICKUP/DELIVERY"
      when upper(accessorial_charge_description) like "%USED%" then "USED"
      when upper(accessorial_charge_description) like "%REDELIVERY%"
        or upper(accessorial_charge_description) like "%RE-DELIVERY%" then "REDELIVERY"
      when upper(accessorial_charge_description) like "%STOP%" then "STOP"
      when upper(accessorial_charge_description) like "%LOADING%" then "LOADING"
      else "OTHER"
      end as accessorial_charge_keyword
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,max(amount_paid) over (partition by cass_shipment_id) as amount_paid
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where concat(origin_state, "-", destination_state) is not null
),
data_with_linehaul_prep1 as (
  select
    *
    ,amount_paid - sum(accessorial_charge_amount) over (partition by cass_shipment_id) as linehaul
  from data_with_lanes
  where accessorial_charge_description not like "%BASE%"
),
data_with_linehaul_prep2 as (
  select
    lane.state_lane
    ,lane.accessorial_charge_description
    ,lane.accessorial_charge_keyword
    ,lane.accessorial_charge_amount
    ,lane.cass_shipment_id
    ,lane.system_type
    ,lane.amount_paid
    ,max(lh.linehaul) over (partition by lane.cass_shipment_id) as linehaul
  from data_with_lanes as lane
  left join data_with_linehaul_prep1 as lh
    on lane.cass_shipment_id = lh.cass_shipment_id
    and lane.accessorial_charge_description = lh.accessorial_charge_description
    and lane.accessorial_charge_amount = lh.accessorial_charge_amount
),
data_with_linehaul as (
  select
    state_lane
    ,accessorial_charge_description
    ,accessorial_charge_keyword
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,amount_paid
    ,if(linehaul is null, amount_paid, linehaul) as linehaul
  from data_with_linehaul_prep2
),
sum as (
  select
    accessorial_charge_keyword
    ,if(accessorial_charge_keyword="BASE", round(sum(linehaul),0), round(sum(accessorial_charge_amount),0)) as keyword_sum
  from data_with_linehaul as lh
  group by accessorial_charge_keyword
),
sum_pct as (
  select
    *
    ,round((keyword_sum / (sum(keyword_sum) over ()))*100, 2) as keyword_sum_pct
  from sum
  group by accessorial_charge_keyword, keyword_sum
)

select * from sum_pct
order by keyword_sum_pct desc
""")

query = query_job.result().to_dataframe()
query

,accessorial_charge_keyword,keyword_sum,keyword_sum_pct
0,BASE,3.213457e+10,67.57
1,FUEL,1.094383e+10,23.01
2,OTHER,2.731580e+09,5.74
3,STOP,3.886919e+08,0.82
4,DETENTION,3.267715e+08,0.69
5,TOLL,3.116735e+08,0.66
6,PICKUP/DELIVERY,2.983150e+08,0.63
7,LABOR,2.116514e+08,0.45
8,LUMPER,1.043527e+08,0.22
9,LOADING,8.777460e+07,0.18


In [32]:
# Comparing accessorial charge amounts by shipper
query_job = client.query("""
with data_with_lanes as (
  select
    concat(origin_state, "-", destination_state) as state_lane
    ,shipper_master_code
    ,count(distinct cass_shipment_id) over (partition by shipper_master_code) as shipper_loads
    ,upper(accessorial_charge_description) as accessorial_charge_description
    ,case
      when upper(accessorial_charge_description) like "%BASE%" then "BASE"
      when upper(accessorial_charge_description) like "%FUEL%" then "FUEL"
      when upper(accessorial_charge_description) like "%TOLL%" then "TOLL"
      when upper(accessorial_charge_description) like "%LABOR%" then "LABOR"
      when upper(accessorial_charge_description) like "%LUMPER%" then "LUMPER"
      when upper(accessorial_charge_description) like "%DETENTION%" then "DETENTION"
      when upper(accessorial_charge_description) like "%PICKUP%"
        or upper(accessorial_charge_description) like "%PICK UP%"
        or upper(accessorial_charge_description) like "%DELIVERY%" then "PICKUP/DELIVERY"
      when upper(accessorial_charge_description) like "%USED%" then "USED"
      when upper(accessorial_charge_description) like "%REDELIVERY%"
        or upper(accessorial_charge_description) like "%RE-DELIVERY%" then "REDELIVERY"
      when upper(accessorial_charge_description) like "%STOP%" then "STOP"
      when upper(accessorial_charge_description) like "%LOADING%" then "LOADING"
      else "OTHER"
      end as accessorial_charge_keyword
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,max(amount_paid) over (partition by cass_shipment_id) as amount_paid
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where concat(origin_state, "-", destination_state) is not null
),
data_with_linehaul_prep1 as (
  select
    *
    ,amount_paid - sum(accessorial_charge_amount) over (partition by cass_shipment_id) as linehaul
  from data_with_lanes
  where accessorial_charge_description not like "%BASE%"
),
data_with_linehaul_prep2 as (
  select
    lane.state_lane
    ,lane.shipper_master_code
    ,lane.shipper_loads
    ,lane.accessorial_charge_description
    ,lane.accessorial_charge_keyword
    ,lane.accessorial_charge_amount
    ,lane.cass_shipment_id
    ,lane.system_type
    ,lane.amount_paid
    ,max(lh.linehaul) over (partition by lane.cass_shipment_id) as linehaul
  from data_with_lanes as lane
  left join data_with_linehaul_prep1 as lh
    on lane.cass_shipment_id = lh.cass_shipment_id
    and lane.accessorial_charge_description = lh.accessorial_charge_description
    and lane.accessorial_charge_amount = lh.accessorial_charge_amount
),
data_with_linehaul as (
  select
    state_lane
    ,shipper_master_code
    ,shipper_loads
    ,accessorial_charge_description
    ,accessorial_charge_keyword
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,amount_paid
    ,if(linehaul is null, amount_paid, linehaul) as linehaul
  from data_with_linehaul_prep2
),
sum as (
  select
    shipper_master_code
    ,shipper_loads
    ,accessorial_charge_keyword
    ,if(accessorial_charge_keyword="BASE", round(sum(linehaul),0), round(sum(accessorial_charge_amount),0)) as keyword_sum
  from data_with_linehaul as lh
  group by shipper_master_code, shipper_loads, accessorial_charge_keyword
)
,
sum_pct as (
  select
    shipper_master_code
    ,shipper_loads
    ,accessorial_charge_keyword
    ,round((keyword_sum / (sum(keyword_sum) over (partition by shipper_master_code)))*100, 2) as keyword_sum_pct
  from sum
  group by shipper_master_code, shipper_loads, accessorial_charge_keyword, keyword_sum
)

select
  *
from sum_pct
where shipper_loads > 1000000
order by accessorial_charge_keyword, keyword_sum_pct
""")

query = query_job.result().to_dataframe()
query

,shipper_master_code,shipper_loads,accessorial_charge_keyword,keyword_sum_pct
0,6463,1488702,BASE,0.00
1,0337,2865325,BASE,0.01
2,6057,1263472,BASE,0.01
3,0333,1358315,BASE,87.76
4,1414,1295772,BASE,90.62
...,...,...,...,...
235,1035,1350226,USED,0.10
236,0153,2614562,USED,0.13
237,1160,2933353,USED,0.14
238,1290,1640618,USED,0.31


In [33]:
# Comparing accessorial charge amounts by lane
query_job = client.query("""
with data_with_lanes as (
  select
    concat(origin_state, "-", destination_state) as state_lane
    ,count(distinct cass_shipment_id) over (partition by concat(origin_state, "-", destination_state)) as lane_loads
    ,shipper_master_code
    ,count(distinct cass_shipment_id) over (partition by shipper_master_code) as shipper_loads
    ,upper(accessorial_charge_description) as accessorial_charge_description
    ,case
      when upper(accessorial_charge_description) like "%BASE%" then "BASE"
      when upper(accessorial_charge_description) like "%FUEL%" then "FUEL"
      when upper(accessorial_charge_description) like "%TOLL%" then "TOLL"
      when upper(accessorial_charge_description) like "%LABOR%" then "LABOR"
      when upper(accessorial_charge_description) like "%LUMPER%" then "LUMPER"
      when upper(accessorial_charge_description) like "%DETENTION%" then "DETENTION"
      when upper(accessorial_charge_description) like "%PICKUP%"
        or upper(accessorial_charge_description) like "%PICK UP%"
        or upper(accessorial_charge_description) like "%DELIVERY%" then "PICKUP/DELIVERY"
      when upper(accessorial_charge_description) like "%USED%" then "USED"
      when upper(accessorial_charge_description) like "%REDELIVERY%"
        or upper(accessorial_charge_description) like "%RE-DELIVERY%" then "REDELIVERY"
      when upper(accessorial_charge_description) like "%STOP%" then "STOP"
      when upper(accessorial_charge_description) like "%LOADING%" then "LOADING"
      else "OTHER"
      end as accessorial_charge_keyword
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,max(amount_paid) over (partition by cass_shipment_id) as amount_paid
  from `freightwaves-data-factory.warehouse.beetlejuice`
  where concat(origin_state, "-", destination_state) is not null
    -- and created_on > "2022-07-31"
),
data_with_linehaul_prep1 as (
  select
    *
    ,amount_paid - sum(accessorial_charge_amount) over (partition by cass_shipment_id) as linehaul
  from data_with_lanes
  where accessorial_charge_description not like "%BASE%"
),
data_with_linehaul_prep2 as (
  select
    lane.state_lane
    ,lane.lane_loads
    ,lane.shipper_master_code
    ,lane.shipper_loads
    ,lane.accessorial_charge_description
    ,lane.accessorial_charge_keyword
    ,lane.accessorial_charge_amount
    ,lane.cass_shipment_id
    ,lane.system_type
    ,lane.amount_paid
    ,max(lh.linehaul) over (partition by lane.cass_shipment_id) as linehaul
  from data_with_lanes as lane
  left join data_with_linehaul_prep1 as lh
    on lane.cass_shipment_id = lh.cass_shipment_id
    and lane.accessorial_charge_description = lh.accessorial_charge_description
    and lane.accessorial_charge_amount = lh.accessorial_charge_amount
),
data_with_linehaul as (
  select
    state_lane
    ,lane_loads
    ,shipper_master_code
    ,shipper_loads
    ,accessorial_charge_description
    ,accessorial_charge_keyword
    ,accessorial_charge_amount
    ,cass_shipment_id
    ,system_type
    ,amount_paid
    ,if(linehaul is null, amount_paid, linehaul) as linehaul
  from data_with_linehaul_prep2
),
sum as (
  select
    state_lane
    ,lane_loads
    ,accessorial_charge_keyword
    ,if(accessorial_charge_keyword="BASE", round(sum(linehaul),0), round(sum(accessorial_charge_amount),0)) as keyword_sum
  from data_with_linehaul as lh
  group by state_lane, lane_loads, accessorial_charge_keyword
)
,
sum_pct as (
  select
    state_lane
    ,lane_loads
    ,accessorial_charge_keyword
    ,round(safe_divide(keyword_sum, (sum(keyword_sum) over (partition by state_lane)))*100, 2) as keyword_sum_pct
  from sum
  group by state_lane, lane_loads, accessorial_charge_keyword, keyword_sum
)

select
  *
from sum_pct
where lane_loads > 500000
order by accessorial_charge_keyword, keyword_sum_pct
""")

query = query_job.result().to_dataframe()
query

,state_lane,lane_loads,accessorial_charge_keyword,keyword_sum_pct
0,MO-MO,629930,BASE,10.82
1,PA-PA,1305556,BASE,21.28
2,ON-ON,1819932,BASE,25.55
3,IL-IL,1605139,BASE,27.94
4,FL-FL,1645340,BASE,38.87
...,...,...,...,...
182,FL-FL,1645340,USED,0.13
183,PA-PA,1305556,USED,0.15
184,TX-TX,4193157,USED,0.18
185,MI-MI,2155473,USED,0.22
